Some notes while converting the comparison plots from the `cea plots-comparisons` to the dashboard.

In [29]:
import cea.config
import cea.inputlocator
reload(cea.config)
config = cea.config.Configuration()
locator = cea.inputlocator.InputLocator(scenario=config.scenario)

In [30]:
import pandas as pd
import os

from cea.analysis.multicriteria.optimization_post_processing.locating_individuals_in_generation_script import \
    get_pointers_to_correct_individual_generation

In [31]:
config.plots_scenario_comparisons.base_scenario

('baseline', None, None)

In [32]:
project = config.plots_scenario_comparisons.project
scenario_baseline = config.plots_scenario_comparisons.base_scenario

scenarios_names = [scenario_baseline] + config.plots_scenario_comparisons.scenarios
network_type = config.plots_scenario_comparisons.network_type

generation_base = scenario_baseline[1]
individual_base = scenario_baseline[2]

scenario_base_path = os.path.join(project, scenario_baseline[0])
scenarios_path = [os.path.join(project, scenario[0]) for scenario in
                  config.plots_scenario_comparisons.scenarios]


TypeError: unbound method decode() must be called with OptimizationIndividualParameter instance as first argument (got NoneType instance instead)

In [28]:
config.plots_scenario_comparisons.scenarios

['baseline|1|ind1', 'baseline|1|ind2']

In [26]:
# scenarios = [[scenario_baseline]+ config.plots_scenario_comparisons.scenarios]
generations = [generation_base] + [scenario[1]
                                   for scenario in config.plots_scenario_comparisons.scenarios]
individuals = [individual_base] + [scenario[2]
                                   for scenario in config.plots_scenario_comparisons.scenarios]
categories = config.plots_scenario_comparisons.categories

generation_pointers, individual_pointers = pointers_all_scenarios(generations, individuals, [
    scenario_base_path] + scenarios_path)  # initialize class
category = "comparisons"

IOError: [Errno 2] No such file or directory: 'C:\\reference-case-cooling\\b\\outputs\\data\\optimization\\master\\CheckPoint_a'

In [21]:
# adaptation of the function `cea.plots.comparisons.main.Plots#preprocessing_demand_scenarios`
def preprocessing_demand_scenarios(self):
    data_processed = pd.DataFrame()
    scenarios_clean = []
    for i, scenario_name in enumerate(self.scenarios_names):
        if scenario_name in scenarios_clean:
            scenario_name = scenario_name + "_duplicated_" + str(i)
        scenarios_clean.append(scenario_name)

    for scenario, scenario_name in zip(self.scenarios, scenarios_clean):
        locator = cea.inputlocator.InputLocator(scenario)
        data_raw = (pd.read_csv(locator.get_total_demand())[self.analysis_fields_demand + ["GFA_m2"]]).sum(axis=0)
        data_raw_df = pd.DataFrame({scenario_name: data_raw}, index=data_raw.index).T
        data_processed = data_processed.append(data_raw_df)
    return data_processed

def pointers_all_scenarios(generations, individuals, scenarios_path):
    generation_pointers = []
    individual_pointers = []
    for scenario_path, generation, individual in zip(scenarios_path, generations, individuals):
        if generation is None or individual is None:
            pointer_gen = "none"
            pointer_ind = "none"
        else:
            locator = cea.inputlocator.InputLocator(scenario_path)
            pointer_gen, pointer_ind = get_pointers_to_correct_individual_generation(generation,
                                                                                     individual, locator)
        generation_pointers.append(pointer_gen)
        individual_pointers.append(pointer_ind)
    return generation_pointers, individual_pointers

In [6]:
preprocessing_demand_scenarios(None)

AttributeError: 'NoneType' object has no attribute 'scenarios_names'

In [12]:
config.plots_scenario_comparisons.scenarios

['C:\\reference-case-cooling\\baseline/1/ind1',
 'C:\\reference-case-cooling\\baseline/1/ind2']

In [9]:
config.plots_scenario_comparisons.base_scenario

'C:\\reference-case-cooling\\baseline'